In [2]:
import os
import random
import numpy as np
import librosa.display
import soundfile as sf
import csv
import json as json

In [3]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [20]:
PATH = "/Users/valleotb/Desktop/Valleotb/sample"
SAVE_PATH = "/Users/valleotb/Desktop/Valleotb/sample_save"
FILE_NAME_PATH = "/Users/valleotb/Desktop/Valleotb/sample_filename"
FILE_COUNT = get_files_count(PATH)

f = open(f'{FILE_NAME_PATH}/metadata.csv','w',newline='')
wr = csv.writer(f)
wr.writerow(['audio_file'])

j = 100000  # 원본 파일 네이밍
z = 200000  # 수정 후 파일 네이밍

for i in range(FILE_COUNT):
    # WAVE_SEQUENCE 에러
    # WAVE_SEQUENCE가 Wav 파일이 바뀔때 마다 0으로 초기화 되어야한다.
    WAVE_SEQUENCE = 0
    # dataload
    j += 1
    WAVEFORM, SR = sf.read(f"{PATH}/{j}.flac")
    WAVEFORM = librosa.util.normalize(WAVEFORM)

    NON_SILENT_TIME = librosa.effects.split(WAVEFORM, top_db=20)
    print(NON_SILENT_TIME)
    print(WAVEFORM.size)

    # os.mkdir(f'{SAVE_PATH}/{z}')
    # MAKE_FOLDER_PATH = f'{SAVE_PATH}/{z}'



    # k 대신 random이 들어가야한다.
    # 근데 그 random이 len(NON_SILENT_TIME) 이상 수로 되면 안된다.
    # 한 샘플에 여러 구간 변형이 있는 것도 괜찮을거 같다.

    # random.randint(0, len(NON_SILENT_TIME)-1)

    if len(NON_SILENT_TIME) > 1 :
        random_num = random.randint(0, len(NON_SILENT_TIME)-1)
        k = NON_SILENT_TIME[random_num]
        if (k[1] - k[0]) > 3000 :


            WAVEFORM, SR = sf.read(f"{PATH}/{j}.flac")
            WAVEFORM = librosa.util.normalize(WAVEFORM)

            print(k[0],k[1])
            N_STEPS = random.randint(1,5)
            WAVEFORM_TRANSFORM = librosa.effects.pitch_shift(WAVEFORM[k[0]:k[1]], sr=SR, n_steps=N_STEPS)
            WAVEFORM_TRANSFORM_MINUS = librosa.effects.pitch_shift(WAVEFORM_TRANSFORM, sr=SR, n_steps=-N_STEPS)
            print("--")
            # Normalize
            WAVEFORM_NOT_TRANSFORM_PART = WAVEFORM[k[0]:k[1]]
            WAVEFORM_DOUBLED = WAVEFORM_NOT_TRANSFORM_PART ** 2
            WAVEFORM_TRANSFORM_DOUBLED = WAVEFORM_TRANSFORM_MINUS ** 2
            a = np.sqrt((WAVEFORM_DOUBLED.sum()) / (WAVEFORM_TRANSFORM_DOUBLED.sum()))

            WAVEFORM_TRANSFORM_FINAL = WAVEFORM_TRANSFORM_MINUS * a

            # Data Process
            WAVEFORM[k[0]:k[1]] = WAVEFORM_TRANSFORM_FINAL

            # Audio를 chunk 단위로 자르기
            rangedata = int(WAVEFORM.size / 1024)
            POINTER = 0


            # 구간의 50% 이상이 포함되었을때만 Labeling 진행한다.

            for e in range(rangedata):
                if POINTER + 1024 <= k[0]:
                    WAVEFORM_CUTTED = WAVEFORM[POINTER:POINTER+1024]
                    sf.write(f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_0.wav', WAVEFORM_CUTTED, SR, subtype="PCM_24")
                    wr.writerow([f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_0.wav'])
                    WAVE_SEQUENCE += 1
                    POINTER += 1025



                elif k[0] < POINTER + 1024 <= k[1]:
                    WAVEFORM_CUTTED = WAVEFORM[POINTER:POINTER+1024]
                    sf.write(f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_1.wav', WAVEFORM_CUTTED, SR, subtype="PCM_24")
                    wr.writerow([f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_1.wav'])
                    WAVE_SEQUENCE += 1
                    POINTER += 1025

                elif POINTER + 1024 > k[1]:
                    WAVEFORM_CUTTED = WAVEFORM[POINTER:POINTER+1024]
                    sf.write(f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_0.wav', WAVEFORM_CUTTED, SR, subtype="PCM_24")
                    wr.writerow([f'{SAVE_PATH}/{z}_{WAVE_SEQUENCE}_0.wav'])
                    WAVE_SEQUENCE += 1
                    POINTER += 1025

            z += 1
            print(f'complete{z}')

f.close()

[[  5632  25088]
 [ 25600  41472]
 [ 43520  44544]
 [ 46592 104960]
 [105472 105984]
 [111616 130560]
 [131072 157184]
 [161792 172544]
 [173056 194560]]
198960
173056 194560
--
complete200001
[[ 6656 53760]
 [57344 71680]]
77200
6656 53760
--
complete200002
[[  5120  24064]
 [ 25600  55808]
 [ 56320  71168]
 [ 72704  80896]
 [ 81408  87040]
 [100352 112640]
 [114176 128000]
 [133632 180224]
 [180736 183808]
 [185856 204800]
 [206336 215040]]
219520
206336 215040
--
complete200003


In [23]:
import pandas as pd

path = "/Users/valleotb/Desktop/Valleotb/sample_metadata/200001_5632_25088.json"
df = pd.read_json(path)
df
# df.columns = ['start_time', 'end_time']

,speech_segments
0,"{'start_time': {'0': 5125}, 'end_time': {'0': ..."
1,"{'start_time': {'1': 6150}, 'end_time': {'1': ..."
2,"{'start_time': {'2': 7175}, 'end_time': {'2': ..."
3,"{'start_time': {'3': 8200}, 'end_time': {'3': ..."
4,"{'start_time': {'4': 9225}, 'end_time': {'4': ..."
5,"{'start_time': {'5': 10250}, 'end_time': {'5':..."
6,"{'start_time': {'6': 11275}, 'end_time': {'6':..."
7,"{'start_time': {'7': 12300}, 'end_time': {'7':..."
8,"{'start_time': {'8': 13325}, 'end_time': {'8':..."
9,"{'start_time': {'9': 14350}, 'end_time': {'9':..."


In [7]:
print(NON_SILENT_TIME)

[[  5120  24064]
 [ 25600  55808]
 [ 56320  71168]
 [ 72704  80896]
 [ 81408  87040]
 [100352 112640]
 [114176 128000]
 [133632 180224]
 [180736 183808]
 [185856 204800]
 [206336 215040]]


In [8]:
erwe = random.randint(0, len(NON_SILENT_TIME)-1)
print(NON_SILENT_TIME[erwe])

[100352 112640]


In [33]:
import json
time_dict = json.load(open(path))
dict(list(time_dict['speech_segments'])[0])['start_time']['0']

5125

In [7]:
WAVEFORM, SR = sf.read(f"{PATH}/{j}.flac")
print(SR)

16000


In [4]:
# i = 100000
filelist = os.listdir(PATH)
print(filelist)
for fn in PATH:
    j = 0
    #i += 1
    # fn_ids = fn.split("-")
    # print(fn)
    flac_fp = os.path.join(PATH, f"{filelist[j]}")
    j += 1
    WAVEFORM, _ = sf.read(flac_fp)
    print(WAVEFORM, f"{j}")

['100001.flac', '100002.flac', '100003.flac']
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037109e-03  3.35693359e-03  2.19726562e-03 ...  6.10351562e-05
 -8.54492188e-04 -2.80761719e-03] 1
[ 2.38037

In [5]:
'''

POINTER = 1024

META_DATA = f"{META_DATA_PATH}/{z}_{k[0]}_{k[1]}.json"
DATA = {}
DATA['speech_segments'] = []

for i in range(WAVEFORM.size / 1024):
    if k[0] > POINTER or k[1] < POINTER :
        DATA['speech_segments'].append({
                    "start_time": POINTER - 1024,
                    "end_time": POINTER
                })
    POINTER += 1024

'''

'\n\nPOINTER = 1024\n\nMETA_DATA = f"{META_DATA_PATH}/{z}_{k[0]}_{k[1]}.json"\nDATA = {}\nDATA[\'speech_segments\'] = []\n\nfor i in range(WAVEFORM.size / 1024):\n    if k[0] > POINTER or k[1] < POINTER :\n        DATA[\'speech_segments\'].append({\n                    "start_time": POINTER - 1024,\n                    "end_time": POINTER\n                })\n    POINTER += 1024\n\n'